In [1]:
from selenium import webdriver
import time
import time
import requests
from pprint import pprint
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import pandas as pd
import re

In [2]:
url="https://www.reddit.com/r/FORTnITE/top/"
browser = webdriver.Chrome()
browser.get(url)
button = browser.find_element_by_tag_name("html")
SCROLL_PAUSE_TIME = 3
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


In [3]:
page=BeautifulSoup(browser.page_source,"lxml")

In [4]:
usernames=[i.get_text() for i in page.find_all("a",class_=re.compile("_2tbHP6Z"))]
links=[i.get("href") for  i in page.find_all("a",class_=re.compile("_2tbHP6Z"))]

In [7]:
z=page.find_all("div",class_="s7z12f0-0 iHQyRe")
upvotes=[i.find("div",class_="_1rZYMD_4xY3gRcSS3p8ODO").get_text() for i in z]
upvotes=[int(re.sub("\.","",re.sub("k","00",i))) for i in upvotes]

In [13]:
base="https://www.reddit.com"
links=[base+i for i in links]
df=pd.DataFrame.from_dict({"username":usernames,"links":links,"upvotes":upvotes},orient="index")
df=df.transpose()


In [29]:
last=df[df.upvotes>50]

In [30]:
def cleaner(x):
    return x[2:]
last.username=last.username.apply(cleaner)
last.tail()

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,username,links,upvotes
30,maxnite,https://www.reddit.comhttps://www.reddit.com/u...,52
34,Bravo4815,https://www.reddit.comhttps://www.reddit.com/u...,64
35,TheRazoredEdge,https://www.reddit.comhttps://www.reddit.com/u...,51
38,mechtanoid,https://www.reddit.comhttps://www.reddit.com/u...,58
40,Kaboinglefop,https://www.reddit.comhttps://www.reddit.com/u...,81


In [32]:
last.to_csv("FORTnITE.csv",index=False)
browser.close()